In [23]:
import pandas as pd

dados = pd.read_parquet("df_com_clusters_atualizados.parquet")

In [24]:
# Lista de colunas a serem removidas
colunas_para_remover = ["release_date", "revenue", "budget", "genres", "original_language", 
                        "production_countries", "spoken_languages", "adult", 
                        "disponibilidade_lucro", "lucro", "runtime", "idade"]

dados = dados.drop(columns = colunas_para_remover)

In [ ]:
import xgboost as xgb
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split

# Separando features (X) e target (y)
X = dados.drop(columns=['classificacao'])
y = dados['classificacao']

# Dividindo os dados em treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

# Criando e treinando o modelo XGBoost
xgb_model = xgb.XGBClassifier(use_label_encoder=False, eval_metric='logloss')
xgb_model.fit(X_train, y_train)

# Fazendo previsões
y_train_pred = xgb_model.predict(X_train)
y_test_pred = xgb_model.predict(X_test)

# Printando o classification report
print("Classification Report para Treino (XGBoost):")
print(classification_report(y_train, y_train_pred))

print("\nClassification Report para Teste (XGBoost):")
print(classification_report(y_test, y_test_pred))


In [ ]:
from sklearn.svm import SVC
from sklearn.metrics import classification_report
import pandas as pd
from sklearn.model_selection import train_test_split
import shap
import numpy as np

# Separando features (X) e target (y)
X = dados.drop(columns=['classificacao'])
y = dados['classificacao']

# Dividindo os dados em treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

# Criando e treinando o modelo SVM
svm_model = SVC(kernel='rbf', C=1.0, gamma='scale')  # Removido probability=True
svm_model.fit(X_train, y_train)

# Fazendo previsões
y_train_pred = svm_model.predict(X_train)
y_test_pred = svm_model.predict(X_test)

# Printando o classification report
print("Classification Report para Treino (SVM):")
print(classification_report(y_train, y_train_pred))

print("\nClassification Report para Teste (SVM):")
print(classification_report(y_test, y_test_pred))

# Aplicando SHAP para explicar as previsões usando KernelExplainer
sample_data = X_train.sample(n=100, random_state=42)  # Amostragem para reduzir custo computacional
explainer = shap.KernelExplainer(svm_model.predict, sample_data)
shap_values = explainer.shap_values(X_test.sample(n=100, random_state=42))

# Plotando a importância das features
shap.summary_plot(shap_values, X_test.sample(n=100, random_state=42))


In [ ]:
selected_countries = ['United States', 'France', 'United Kingdom', 'Germany', 
                      'Canada', 'Japan', 'China', 'India', 'Italy', 'Spain', "United States of America"]